# Getting Soil Grid Data

In [1]:
import pandas as pd
import requests
import json
import time
import random
import os
import concurrent.futures
from tqdm import tqdm
import threading

# Add a lock to prevent race conditions when saving data
save_lock = threading.Lock()

def get_soilgrids_point(lon, lat, point_idx, properties=None, max_retries=3):
    """Get SoilGrids data for a single point with correct field mapping"""
    if properties is None:
        properties = ['soc', 'clay', 'sand', 'silt', 'bdod', 'phh2o']
        
    url = "https://rest.isric.org/soilgrids/v2.0/properties/query"
    params = {
        'lon': lon,
        'lat': lat,
        'property': properties,
        'depth': ['0-5cm', '5-15cm', '15-30cm'],
        'value': ['mean']
    }
    
    for retry in range(max_retries):
        try:
            response = requests.get(url, params=params)
            
            # Handle rate limiting
            if response.status_code == 429:
                wait_time = 15 + random.random() * 15
                #print(f"Rate limited for point {point_idx}, waiting {wait_time:.1f} seconds")
                time.sleep(wait_time)
                continue
                
            if response.status_code == 200:
                data = response.json()
                
                # Start with basic info
                result = {'point_index': point_idx, 'lon': lon, 'lat': lat}
                
                # Extract data using the correct field structure
                if 'properties' in data and 'layers' in data['properties']:
                    for layer in data['properties']['layers']:
                        # Get property name
                        prop_name = layer.get('name', 'unknown')
                        
                        for depth in layer.get('depths', []):
                            # Get depth label (which is the string format we need)
                            depth_label = depth.get('label', 'unknown')
                            
                            # Clean the depth label for column naming
                            clean_depth = depth_label.replace('-', '_to_')
                            
                            # Extract values
                            for value_type, value in depth.get('values', {}).items():
                                column_name = f"{prop_name}_{clean_depth}_{value_type}"
                                result[column_name] = value
                
                # Debug print to verify data is being captured correctly
                #print(f"Retrieved data for point {point_idx}: {lon}, {lat}")
                return result
            else:
                #print(f"Error for point {point_idx}: Status code {response.status_code}")
                if retry < max_retries - 1:
                    wait_time = 10 * (retry + 1)
                    #print(f"Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)
                else:
                    return {'point_index': point_idx, 'lon': lon, 'lat': lat, 
                            'error': f"Status {response.status_code}"}
        
        except Exception as e:
            #print(f"Exception for point {point_idx}: {str(e)}")
            if retry < max_retries - 1:
                wait_time = 10 * (retry + 1)
                #print(f"Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                return {'point_index': point_idx, 'lon': lon, 'lat': lat, 
                        'error': f"Exception: {str(e)}"}
    
    return {'point_index': point_idx, 'lon': lon, 'lat': lat, 
            'error': "Max retries reached"}

def process_point(args):
    """Wrapper function for concurrent processing"""
    lon, lat, idx, properties = args
    # Add jitter to avoid all workers hitting the API simultaneously
    time.sleep(random.random() * 2)
    return get_soilgrids_point(lon, lat, idx, properties)

def save_checkpoint(results, filename, verbose=False):
    """Save results to a checkpoint file using a lock to prevent race conditions"""
    with save_lock:
        try:
            df_results = pd.DataFrame(results)
            # First write to a temporary file, then rename to avoid partial writes
            temp_file = f"{filename}.temp"
            df_results.to_csv(temp_file, index=False)
            os.replace(temp_file, filename)
            if verbose:
                print(f"Saved checkpoint with {len(results)} points to {filename}")
        except Exception as e:
            print(f"Error saving checkpoint: {str(e)}")

def get_soilgrids_parallel(coordinates_df, num_workers=4, lon_col='GPS_LONG', lat_col='GPS_LAT', 
                           properties=None, cache_file='soilgrids_parallel.csv',
                           checkpoint_interval=10, debug=False):
    """
    Retrieve soil data for multiple points in parallel using multiple workers
    
    Args:
        coordinates_df: DataFrame with coordinates
        num_workers: Number of parallel workers (default: 4)
        lon_col: Column name for longitude
        lat_col: Column name for latitude
        properties: List of SoilGrids properties to retrieve
        cache_file: Output file name
        checkpoint_interval: Save intermediate results every N points
        debug: Enable additional debug output
    """
    if properties is None:
        properties = ['soc', 'clay', 'sand', 'silt', 'bdod', 'phh2o']
    
    # Print the input data to verify it's correct
    if debug:
        print("Input coordinate data sample:")
        print(coordinates_df.head())
        print(f"Longitude column: {lon_col}, Latitude column: {lat_col}")
    
    # Check for existing cache to resume from
    results = []
    
    if os.path.exists(cache_file):
        try:
            existing_df = pd.read_csv(cache_file)
            if len(existing_df) > 0:
                results = existing_df.to_dict('records')
                processed_indices = set(existing_df['point_index'].unique())
                print(f"Found {len(processed_indices)} already processed points in {cache_file}")
                coordinates_df = coordinates_df[~coordinates_df.index.isin(processed_indices)]
                print(f"Remaining points to process: {len(coordinates_df)}")
        except Exception as e:
            print(f"Error reading existing cache: {str(e)}. Starting from scratch.")
    
    if len(coordinates_df) == 0:
        print("All points already processed!")
        return pd.DataFrame(results)
    
    # Prepare arguments for parallel processing
    args_list = []
    for idx, row in coordinates_df.iterrows():
        # Verify and clean coordinate values
        try:
            lon = float(row[lon_col])
            lat = float(row[lat_col])
            args_list.append((lon, lat, idx, properties))
            if debug and len(args_list) <= 5:
                print(f"Prepared point {idx}: lon={lon}, lat={lat}")
        except (ValueError, TypeError) as e:
            print(f"Error with coordinates at index {idx}: {e}")
            print(f"Row data: {row}")
    
    print(f"Processing {len(args_list)} points with {num_workers} workers")
    
    completed_count = 0
    
    # Use ThreadPoolExecutor for parallel HTTP requests
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
        # Submit all tasks
        future_to_args = {executor.submit(process_point, args): args for args in args_list}
        
        # Use tqdm for a progress bar
        for future in tqdm(concurrent.futures.as_completed(future_to_args), total=len(args_list)):
            args = future_to_args[future]
            point_idx = args[2]
            
            try:
                result = future.result()
                if result:
                    results.append(result)
                    completed_count += 1
                    
                    # Save intermediate results periodically
                    if completed_count % checkpoint_interval == 0:
                        save_checkpoint(results, cache_file)
                
            except Exception as e:
                print(f"\nError processing point {point_idx}: {str(e)}")
    
    # Save final results
    save_checkpoint(results, cache_file, verbose=False)
    
    # Verify the final output
    try:
        final_df = pd.read_csv(cache_file)
        print(f"Final output has {len(final_df)} rows and {len(final_df.columns)} columns")
        print("Column names:", final_df.columns.tolist())
        print("First few rows:")
        print(final_df.head())
    except Exception as e:
        print(f"Error verifying final output: {str(e)}")
    
    return pd.DataFrame(results)

# Example usage:
# df = pd.read_csv('coordinates.csv', index_col=0)  # Set the first column as index if that's your point_index
# results = get_soilgrids_parallel(df, num_workers=4)

In [2]:
# Load your data
target_raw = pd.read_csv('data/France_lab.csv')
long_lat = target_raw[['GPS_LONG', 'GPS_LAT']]
get_soilgrids_parallel(long_lat, num_workers=4, properties=['soc', 'clay', 'sand', 'silt', 'bdod', 'phh2o'], debug = False)

Found 670 already processed points in soilgrids_parallel.csv
Remaining points to process: 2137
Processing 2137 points with 4 workers


100%|██████████| 2137/2137 [1:41:01<00:00,  2.84s/it]  


Final output has 2807 rows and 22 columns
Column names: ['point_index', 'lon', 'lat', 'bdod_0_to_5cm_mean', 'bdod_5_to_15cm_mean', 'bdod_15_to_30cm_mean', 'clay_0_to_5cm_mean', 'clay_5_to_15cm_mean', 'clay_15_to_30cm_mean', 'phh2o_0_to_5cm_mean', 'phh2o_5_to_15cm_mean', 'phh2o_15_to_30cm_mean', 'sand_0_to_5cm_mean', 'sand_5_to_15cm_mean', 'sand_15_to_30cm_mean', 'silt_0_to_5cm_mean', 'silt_5_to_15cm_mean', 'silt_15_to_30cm_mean', 'soc_0_to_5cm_mean', 'soc_5_to_15cm_mean', 'soc_15_to_30cm_mean', 'error']
First few rows:
   point_index       lon        lat  bdod_0_to_5cm_mean  bdod_5_to_15cm_mean  \
0            1  4.584692  45.816720               125.0                134.0   
1            0  4.680379  45.893933               128.0                138.0   
2            3  4.601575  45.908022               133.0                140.0   
3            2  4.671533  45.983716               129.0                139.0   
4            6  4.439863  46.224665               102.0                116.

,point_index,lon,lat,bdod_0_to_5cm_mean,bdod_5_to_15cm_mean,bdod_15_to_30cm_mean,clay_0_to_5cm_mean,clay_5_to_15cm_mean,clay_15_to_30cm_mean,phh2o_0_to_5cm_mean,...,sand_0_to_5cm_mean,sand_5_to_15cm_mean,sand_15_to_30cm_mean,silt_0_to_5cm_mean,silt_5_to_15cm_mean,silt_15_to_30cm_mean,soc_0_to_5cm_mean,soc_5_to_15cm_mean,soc_15_to_30cm_mean,error
0,1,4.584692,45.816720,125.0,134.0,141.0,250.0,270.0,302.0,58.0,...,375.0,366.0,366.0,375.0,364.0,332.0,489.0,249.0,246.0,NaN
1,0,4.680379,45.893933,128.0,138.0,141.0,303.0,319.0,338.0,62.0,...,263.0,243.0,275.0,434.0,438.0,387.0,467.0,254.0,182.0,NaN
2,3,4.601575,45.908022,133.0,140.0,144.0,247.0,268.0,288.0,60.0,...,334.0,320.0,329.0,418.0,412.0,383.0,401.0,289.0,153.0,NaN
3,2,4.671533,45.983716,129.0,139.0,143.0,249.0,254.0,300.0,63.0,...,305.0,295.0,313.0,446.0,451.0,387.0,422.0,271.0,154.0,NaN
4,6,4.439863,46.224665,102.0,116.0,121.0,188.0,172.0,205.0,52.0,...,505.0,514.0,494.0,306.0,314.0,301.0,786.0,620.0,239.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2802,2803,5.058028,45.713629,131.0,147.0,150.0,195.0,196.0,215.0,63.0,...,376.0,366.0,375.0,429.0,439.0,410.0,448.0,173.0,232.0,NaN
2803,2806,4.784826,45.881063,128.0,137.0,141.0,181.0,164.0,204.0,60.0,...,323.0,314.0,344.0,496.0,522.0,452.0,430.0,167.0,187.0,NaN
2804,2805,4.381513,45.788303,124.0,135.0,137.0,201.0,193.0,213.0,58.0,...,434.0,437.0,434.0,366.0,369.0,353.0,541.0,294.0,260.0,NaN
2805,2800,4.718750,45.498638,119.0,129.0,135.0,169.0,159.0,212.0,58.0,...,467.0,473.0,440.0,364.0,368.0,348.0,545.0,215.0,212.0,NaN


## Update

In [8]:
def update_missing_soilgrids_data(csv_file, output_file=None, max_retries=5, delay_between_retries=20):
    """
    Update missing data in a SoilGrids CSV file
    
    Args:
        csv_file: Path to the CSV file with missing data
        output_file: Path to save the updated CSV (default: overwrite input file)
        max_retries: Maximum number of retries for failed API calls
        delay_between_retries: Delay in seconds between retries
        
    Returns:
        DataFrame with the updated data
    """
    import pandas as pd
    import time
    import random
    import numpy as np
    
    if output_file is None:
        output_file = csv_file
    
    # Load the CSV file and force column types
    print(f"Loading data from {csv_file}...")
    df = pd.read_csv(csv_file, header=None)
    
    # Determine data types for all columns
    dtypes = df.dtypes
    print(f"Column data types: {dtypes}")
    
    # Identify rows with missing data (rows with mostly empty values)
    # Consider both NaN values and empty strings as missing
    missing_mask = ((df.iloc[:, 3:].isna()) | (df.iloc[:, 3:] == "")).sum(axis=1) > (df.shape[1] - 3) * 0.5
    missing_indices = df[missing_mask].index
    
    print(f"Found {len(missing_indices)} rows with missing data")
    
    if len(missing_indices) == 0:
        print("No missing data to update!")
        return df
    
    # Prepare a results list to store updated rows
    updated_rows = []
    
    # Process each row with missing data
    for idx in missing_indices:
        row = df.iloc[idx]
        point_idx = row[0]
        lon = row[1]
        lat = row[2]
        
        print(f"Processing missing data for point {point_idx} at coordinates {lon}, {lat}")
        
        # Make API call with retries
        for retry in range(max_retries):
            try:
                result = get_soilgrids_point(lon, lat, point_idx)
                
                if 'error' in result:
                    print(f"Attempt {retry+1}/{max_retries} failed: {result.get('error')}")
                    
                    # If we've reached the max retries, save what we have
                    if retry == max_retries - 1:
                        print(f"Failed to update point {point_idx} after {max_retries} attempts")
                        break
                    
                    # Wait before retrying
                    sleep_time = delay_between_retries + random.random() * 10
                    print(f"Retrying in {sleep_time:.1f} seconds...")
                    time.sleep(sleep_time)
                    continue
                
                # Create a new row with the correct data types
                updated_row = row.copy()
                
                # Set the basic fields (point_idx, lon, lat)
                # Convert to the same type as the original DataFrame to avoid warnings
                updated_row[0] = point_idx  # This should already be the correct type
                updated_row[1] = lon        # This should already be the correct type
                updated_row[2] = lat        # This should already be the correct type
                
                # Map the result fields to the appropriate columns in the dataframe
                soil_properties = ['soc', 'clay', 'sand', 'silt', 'bdod', 'phh2o']
                depths = ['0_to_5cm', '5_to_15cm', '15_to_30cm']
                
                # Assuming the columns in the original DataFrame follow this order:
                column_idx = 3  # Start after point_idx, lon, lat
                for prop in soil_properties:
                    for depth in depths:
                        column_name = f"{prop}_{depth}_mean"
                        if column_name in result and column_idx < len(df.columns):
                            # Try to match the data type
                            value = result[column_name]
                            if pd.api.types.is_float_dtype(dtypes[column_idx]):
                                value = float(value) if value is not None else np.nan
                            elif pd.api.types.is_integer_dtype(dtypes[column_idx]):
                                value = int(value) if value is not None else np.nan
                            updated_row[column_idx] = value
                        column_idx += 1
                
                # Update the DataFrame
                df.iloc[idx] = updated_row
                print(f"Successfully updated point {point_idx}")
                
                # Add a small delay to avoid rate limiting
                time.sleep(2 + random.random() * 3)
                break
                
            except Exception as e:
                print(f"Error updating point {point_idx}: {str(e)}")
                
                if retry < max_retries - 1:
                    sleep_time = delay_between_retries + random.random() * 10
                    print(f"Retrying in {sleep_time:.1f} seconds...")
                    time.sleep(sleep_time)
                else:
                    print(f"Failed to update point {point_idx} after {max_retries} attempts")
    
    # Save the updated DataFrame
    print(f"Saving updated data to {output_file}...")
    df.to_csv(output_file, index=False, header=False)
    
    return df

In [9]:
# Update the missing data
updated_df = update_missing_soilgrids_data(
    csv_file='data/soilgrids_parallel.csv',
    output_file='data/soilgrids_updated.csv',
    max_retries=5,
    delay_between_retries=20
)

Loading data from data/soilgrids_parallel.csv...
Column data types: 0     object
1     object
2     object
3     object
4     object
5     object
6     object
7     object
8     object
9     object
10    object
11    object
12    object
13    object
14    object
15    object
16    object
17    object
18    object
19    object
20    object
21    object
dtype: object
Found 103 rows with missing data
Processing missing data for point 155 at coordinates 0.669178, 49.855175
Successfully updated point 155
Processing missing data for point 191 at coordinates 2.743706, 48.544291
Successfully updated point 191
Processing missing data for point 190 at coordinates 2.557593, 48.513858
Successfully updated point 190
Processing missing data for point 216 at coordinates 3.131174, 48.695394
Successfully updated point 216
Processing missing data for point 250 at coordinates -0.054073, 46.837853
Successfully updated point 250
Processing missing data for point 297 at coordinates 2.737896, 49.699723
Succe